

## Challenge of processing large amounts of data
<ul>
<li>How to process is quickly?
<li>So how do we go about making the problem map so that it can be distributed computation?
<li> Distributed/Parrallel Programming is hard
</ul>

##Mapreduce addresses all the challengs
<ul>
<li>Google's computational/data manipulation model
<li> Elegant way to work with big data
</ul>

## Map Reduce and the New Software Stack
<ul>
<li> Covered in Chapter 2 in the Ullman book
<li> Python support packages https://pypi.python.org/pypi/mrjob
<li> map() and reduce() are basic built in functions in python https://docs.python.org/2/library/functions.html
</ul>


## Built-in Python Functional Programming Tools
https://docs.python.org/2/tutorial/datastructures.html

### Python Map Function
For each value in a sequence, process each one and output a new result for each element.

In [6]:
def cube(x): return x*x*x

map(cube,range(1,11))

[1, 8, 27, 64, 125, 216, 343, 512, 729, 1000]

If there are more parameters then each could be an array, and they are applied together one element at a time

In [7]:
seq = range(8)
def add(x,y): return x+y
map(add, seq,seq)

[0, 2, 4, 6, 8, 10, 12, 14]

## Python Reduce Function
For an array passed in, compute a single result.
So a reduce function always has two paramters to carry the result foward with the next element in the sequence.
The operation starts by using the first two values then passes the result with the next element to the function...

In [8]:
result = map(add, seq,seq)
reduce(add, result)  # adding each element of the result together

56

If there is only one value in a sequence then that element is returned; if the sequence is empty, an exception is raised.

##Word Count using python in a map reduce manner
Using the Canterbury Corpus Test file from http://compression.ca/act/files/canterbury.zip,
We will attempt to count the each unique word.


In [11]:
import re
import pandas as pd
import numpy as np
aliceFile = open('data/canterbury/alice29.txt','r')
map1=[]

WORD_RE = re.compile(r"[\w']+")

# Create the map of words with prelminary counts
for line in aliceFile:
    for w in WORD_RE.findall(line):
        map1.append([w,1])

#sort the map            
map2 = sorted(map1)

#Separate the map into groups by the key values
df = pd.DataFrame(map2)

uniquewords = df[0].unique()
DataFrameDict = {elem : pd.DataFrame for elem in uniquewords}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df[:][df[0] == key]


In [13]:
def wordcount(x,y):
    x[1] = x[1] + y[1]
    return x

#Add up the counts using reduce

for uw in uniquewords:
    uarray = np.array(DataFrameDict[uw])
    print reduce(wordcount,uarray)


["'" 1091L]
["'TIS" 1L]
["'Tis" 2L]
["'em" 3L]
["'tis" 2L]
['2' 1L]
['9' 1L]
['A' 17L]
['ADVENTURES' 1L]
["ALICE'S" 3L]
['ALL' 4L]
['AND' 3L]
['ARE' 6L]
['AT' 1L]
['Ada' 1L]
['Adventures' 2L]
['Advice' 1L]
['After' 6L]
['Ah' 5L]
['Ahem' 1L]
['Alas' 1L]
['Alice' 386L]
["Alice's" 9L]
['All' 5L]
['Allow' 1L]
['Always' 1L]
['Ambition' 1L]
['An' 5L]
['And' 67L]
['Ann' 4L]
['Antipathies' 1L]
['Anything' 1L]
['Are' 4L]
['Arithmetic' 1L]
['As' 17L]
['At' 9L]
['Atheling' 1L]
['Australia' 1L]
['BE' 1L]
['BEE' 1L]
['BEFORE' 1L]
['BEG' 1L]
['BEST' 2L]
['BOOTS' 1L]
['BUSY' 1L]
['Back' 1L]
['Be' 2L]
['Beau' 4L]
['Beautiful' 5L]
['Because' 1L]
['Before' 1L]
['Begin' 1L]
['Behead' 1L]
['Besides' 1L]
['Between' 1L]
['Bill' 12L]
["Bill's" 4L]
['Birds' 1L]
['Boots' 1L]
['Brandy' 1L]
['Bring' 1L]
['But' 37L]
['By' 4L]
['C' 1L]
['CAN' 4L]
['CHAPTER' 12L]
['CHORUS' 2L]
['COULD' 4L]
['COURT' 1L]
['CURTSEYING' 1L]
['Call' 4L]
['Can' 1L]
["Can't" 1L]
['Canary' 1L]
['Canterbury' 1L]
['Carroll' 1L]
['Cat' 24L]
[

## MapReduce using MRJOB
Find documentation for MRJOB at https://pythonhosted.org/mrjob/
A framework that allows you to do mapreduce jobs without HADOOP but will run the same jobs in an hadoop environment.
( DUMBO and Pydoop give you lower level access to HADOOP)

Before using it for the first time install this package using : <code> pip install mrjob </code> 

If that does not work use the alternatives in provided on https://pythonhosted.org/mrjob/guides/quickstart.html#installation

%%writefile myfile.py

write/save cell contents into myfile.py (use -a to append). Another alias: %%file myfile.py
%run myfile.py

run myfile.py and output results in the current cell
%load myfile.py

load "import" myfile.py into the current cell

In [ ]:
# %load code/MRWordFrequencyCount.py
from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):
    
    def mapper(self, _ , line):
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1
        
    def reducer(self, key, values):
        yield key, sum(values)
        
if __name__ == '__main__':
    MRWordFrequencyCount.run()

In [1]:
%run code/MRWordFrequencyCount.py  ./.mrjob.conf data/canterbury/alice29.txt

no configs found; falling back on auto-configuration
INFO:mrjob.conf:no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
INFO:mrjob.conf:no configs found; falling back on auto-configuration
creating tmp directory c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000
INFO:mrjob.runner:creating tmp directory c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000

PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or set up mrjob.conf as described at https://pythonhosted.org/mrjob/whats-new.html#ready-for-strict-protocols

writing to c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000\step-0-mapper_part-00000
INFO:mrjob.sim:writing to c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000\step-0-mapper_part-00000
writing to c:\users\ps\appdata\local\temp\MRWor

"chars"	148685
"lines"	12
"words"	26474


removing tmp directory c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000
INFO:mrjob.runner:removing tmp directory c:\users\ps\appdata\local\temp\MRWordFrequencyCount.PS.20150922.022135.283000


In [ ]:
# MRJOB Word count function

In [ ]:
# %load code\MRWordFreqCount.py
from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"[\w']+")


class MRWordFreqCount(MRJob):

    def mapper(self, _, line):
        for word in WORD_RE.findall(line):
            yield word.lower(), 1

    def combiner(self, word, counts):
        yield word, sum(counts)

    def reducer(self, word, counts):
        yield word, sum(counts)


if __name__ == '__main__':
    MRWordFreqCount.run()

In [14]:
%run code/MRWordFreqCount.py data/canterbury/alice29.txt

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
INFO:mrjob.conf:no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
INFO:mrjob.conf:no configs found; falling back on auto-configuration
creating tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
creating tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
creating tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
INFO:mrjob.runner:creating tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000



PLEASE NOTE: Starting in mrjob v0.5.0, protocols will be strict by default. It's recommended you run your job with --strict-protocols or se

"'"	1091
"'em"	3
"'tis"	5
"_i_"	2
"2"	1
"9"	1
"a"	632
"abide"	1
"able"	1
"about"	94
"above"	3
"absence"	1
"absurd"	2
"acceptance"	1
"accident"	2
"accidentally"	1
"account"	1
"accounting"	1
"accounts"	1
"accusation"	1
"accustomed"	1
"ache"	1
"across"	5
"act"	1
"actually"	1
"ada"	1
"added"	23
"adding"	1
"addressed"	2
"addressing"	1
"adjourn"	1
"adoption"	1
"advance"	3
"advantage"	3
"adventures"	7
"advice"	2
"advisable"	2
"advise"	1
"affair"	1
"affectionately"	1
"afford"	1
"afore"	1
"afraid"	12
"after"	43
"afterwards"	2
"again"	82
"against"	9
"age"	4
"ago"	2
"agony"	1
"agree"	2
"ah"	5
"ahem"	1
"air"	15
"airs"	1
"alarm"	2
"alarmed"	1
"alas"	4
"alice's"	12
"alice"	386
"alive"	3
"all"	182
"allow"	3
"almost"	6
"alone"	4
"along"	6
"aloud"	5
"already"	2
"also"	2
"altered"	1
"alternately"	1
"altogether"	5
"always"	13
"am"	15
"ambition"	1
"among"	12
"an"	57
"ancient"	1
"and"	872
"anger"	2
"angrily"	9
"angry"	5
"animal's"	1
"animal"	1
"animals"	4
"ann"	4
"annoy"	1
"annoyed"	1
"another"	22
"answer"

removing tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
removing tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
removing tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000
INFO:mrjob.runner:removing tmp directory c:\users\ps\appdata\local\temp\MRWordFreqCount.PS.20150922.023122.602000


In [ ]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

from __future__ import print_function

import sys
from operator import add

from pyspark import SparkContext


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: wordcount <file>", file=sys.stderr)
        exit(-1)
    sc = SparkContext(appName="PythonWordCount")
    lines = sc.textFile(sys.argv[1], 1)
    counts = lines.flatMap(lambda x: x.split(' ')) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add)
    output = counts.collect()
    for (word, count) in output:
        print("%s: %i" % (word, count))

    sc.stop()